In [1]:
import Graph
import pandas as pd
import json
from ast import literal_eval
import networkx as nx
from tqdm import tqdm
import WordPreprocessing
import math
import community

import pickle
import random
from gensim.models.keyedvectors import Word2VecKeyedVectors
s=WordPreprocessing.sentence()

In [2]:
def top_n(count_dict, reverse, n=3):
    return dict(sorted(count_dict.items(), reverse=reverse, key=lambda x: x[1])[:n])

In [3]:
def make_info():
    title=pd.read_csv('./EndData/total_review_end.csv',index_col=0)['title']
    nfid=pd.read_csv('./EndData/total_review_end.csv',index_col=0)['nfid']
    return pd.concat([nfid,title],axis=1)

In [4]:
def recommend(mbti,title):
    gm = Graph.MakeGraph(mbti_data=1)
    g = pickle.load(open(f'../../../../바탕화면/데이터/final_graph/{mbti}.pickle', 'rb'))
    model = Word2VecKeyedVectors.load(f'../../../../바탕화면/데이터/model/{mbti}.model')

    dic = {}
    for n,ti in tqdm(enumerate(list(title))):
        # if n==10: break
        w=0
        weight = []
        for tar in [f'{mbti.lower()}']:
            c=1
            d = nx.shortest_path(g,source=tar,target=ti,weight='distance')
            for i in range(0,len(d)-1):
                w1 = model.similarity(str(d[i]),str(d[i+1]))
                
                w2 = model.similarity(f'{tar}',str(d[i+1]))
                w3 = model.distance(str(d[i]),str(d[i+1]))
        
                if 'p' in tar:
                    w += (w1+w2)*w3
                else:
                    w += (w1)*w3
                c+=0.5
            weight.append(round(w/c,3))
        dic[ti]=max(weight)
    return dic

In [6]:
def recommend_keyword(mbti,title,keyword):
    gm = Graph.MakeGraph(mbti_data=1)
    g = pickle.load(open(f'../../../../바탕화면/데이터/final_graph/{mbti}.pickle', 'rb'))
    model = Word2VecKeyedVectors.load(f'../../../../바탕화면/데이터/model/{mbti}.model')
    mbti = mbti.lower()
    dic={}
    movie_list = []
    
    for n,ti in tqdm((enumerate(title))):
        w=0
        c=1
        weight = []
        
        
        d = nx.shortest_path(g,f'{mbti}',f'{keyword}',weight='distance')
        d.extend(nx.shortest_path(g,f'{keyword}',f'{ti}',weight='distance')[1:])
        for i in range(0,len(d)-1):
            w1 = model.similarity(str(d[i]),str(d[i+1]))
            w2 = model.similarity(f'{mbti}',str(d[i+1]))
            w3 = model.distance(str(d[i]),str(d[i+1]))

            w += (w1+w2)*w3
            c*=1.5
        weight.append(round(w/c,3))
        dic[ti]=max(weight)
    movie_list.extend(list(top_n(dic,True,20).keys()))
    return movie_list
    


In [7]:
def recommender_sysyem(mbti,info_df,keyword=None):
    if keyword==None:
        movie_list = recommend(f'{mbti}',info_df['title'])
        top_40 = list(top_n(movie_list,True,40).keys())
        sample_list = random.sample(top_40,5)
    else:
        movie_list = [] 
        for k in keyword:
            if k!=None:
                movie_list.extend(recommend_keyword(f'{mbti}',info_df['title'],k))
        sample_list = random.sample(sum([],movie_list),5)
        
    return list(info_df[info_df['title'].isin(sample_list)]['nfid'])
    

In [9]:
info_df = pd.read_csv('../../../../바탕화면/데이터/info_df.csv')

In [10]:
keyword=['잔인','메시지']

In [11]:
l=recommender_sysyem('INTP',info_df,keyword)

2023-08-09 20:05:31,985 : INFO : loading Word2VecKeyedVectors object from ../../../../바탕화면/데이터/model/INTP.model
2023-08-09 20:05:32,074 : INFO : setting ignored attribute vectors_norm to None
2023-08-09 20:05:32,074 : INFO : loaded ../../../../바탕화면/데이터/model/INTP.model
418it [00:08, 50.23it/s]
2023-08-09 20:05:40,460 : INFO : loading Word2VecKeyedVectors object from ../../../../바탕화면/데이터/model/INTP.model
2023-08-09 20:05:40,556 : INFO : setting ignored attribute vectors_norm to None
2023-08-09 20:05:40,557 : INFO : loaded ../../../../바탕화면/데이터/model/INTP.model
418it [00:08, 47.74it/s]


In [12]:
l

[70241118, 80014749, 81321999, 80174974, 70270364]

In [13]:
l=recommender_sysyem('INTP',info_df)

2023-08-09 20:06:47,935 : INFO : loading Word2VecKeyedVectors object from ../../../../바탕화면/데이터/model/INTP.model
2023-08-09 20:06:47,973 : INFO : setting ignored attribute vectors_norm to None
2023-08-09 20:06:48,036 : INFO : loaded ../../../../바탕화면/데이터/model/INTP.model
418it [00:02, 181.38it/s]


In [14]:
l

[70264888, 70012909, 81008221, 70305907, 81121351]

# 노드 삭제

In [377]:
info_df.index=info_df['nfid']

In [368]:
nfid_list

['81161927',
 '81254340',
 '80176842',
 '81497886',
 '81078819',
 '81354041',
 '70142405',
 '81671553',
 '81106901']

In [371]:
info_df.loc[nfid_list]

,nfid,title
nfid,,
81161927,81161927,리얼
81254340,81254340,데이트 앱 사기: 당신을 노린다
80176842,80176842,아는 형님
81497886,81497886,샤펠 사단 - 어스퀘이크: 전설의 코미디
81078819,81078819,브레이킹 배드 무비: 엘 카미노
81354041,81354041,사이버 지옥: n번방을 무너뜨려라
70142405,70142405,아바타 아앙의 전설
81671553,81671553,압꾸정
81106901,81106901,블랙핑크: 세상을 밝혀라


In [372]:
info_df.drop(nfid_list,axis=0,inplace=True)

In [379]:
info_df.index=range(len(info_df))

In [382]:
info_df[info_df['title']=='마더']

,nfid,title
11,70118372,마더


In [384]:
info_df.to_csv('info_df.csv',index=False)

In [7]:
pd.read_csv('./info_df.csv')

,nfid,title
0,80987077,시그널
1,70020509,히트
2,80113804,아가씨
3,70241118,도가니
4,60002360,쥬라기 공원
...,...,...
413,80100172,다크
414,70032593,유령신부
415,70155547,스폰지밥 네모바지
416,80010655,블러드라인


In [376]:
duplicate=[80177458,81292901]

In [378]:
info_df.drop(duplicate,axis=0,inplace=True)

In [12]:
mbti_list = ['ISTJ','ISFJ','INFJ','INTJ',
                        'ISTP','ISFP','INFP','INTP',
                        'ESTP','ESFP','ENFP','ENTP',
                        'ESTJ','ESFJ','ENFJ','ENTJ']

In [16]:
for mbti in mbti_list:
    for name in name_list:
        gm = Graph.MakeGraph(mbti_data=1)
        g = pickle.load(open(f'./network/final_graph/sample2/{mbti}.pickle', 'rb'))
        # model = Word2VecKeyedVectors.load(f'./network/model/sample2/{mbti}.model')
        try:
            g.remove_node(name)
        except:
            print(name,mbti)
        gm.save_graph(g,f'./network/final_graph/{mbti}')

리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ISTJ
압꾸정,블랙핑크: 세상을 밝혀라 ISTJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ISFJ
압꾸정,블랙핑크: 세상을 밝혀라 ISFJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 INFJ
압꾸정,블랙핑크: 세상을 밝혀라 INFJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 INTJ
압꾸정,블랙핑크: 세상을 밝혀라 INTJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ISTP
압꾸정,블랙핑크: 세상을 밝혀라 ISTP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ISFP
압꾸정,블랙핑크: 세상을 밝혀라 ISFP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 INFP
압꾸정,블랙핑크: 세상을 밝혀라 INFP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 INTP
압꾸정,블랙핑크: 세상을 밝혀라 INTP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ESTP
압꾸정,블랙핑크: 세상을 밝혀라 ESTP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ESFP
압꾸정,블랙핑크: 세상을 밝혀라 ESFP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ENFP
압꾸정,블랙핑크: 세상을 밝혀라 ENFP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ENTP
압꾸정,블랙핑크: 세상을 밝혀라 ENTP
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ESTJ
압꾸정,블랙핑크: 세상을 밝혀라 ESTJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ESFJ
압꾸정,블랙핑크: 세상을 밝혀라 ESFJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ENFJ
압꾸정,블랙핑크: 세상을 밝혀라 ENFJ
리얼, 데이트 앱 사기: 당신을 노린다,아는 형님 ENTJ
압꾸정,블랙핑크: 세상을 밝혀라 ENTJ


In [14]:
mbti

'ISTJ'

In [10]:
g = pickle.load(open(f'./network/final_graph/sample2/INTJ.pickle', 'rb'))

In [ ]:
g.remove_node

In [8]:
name_list=['리얼, 데이트 앱 사기: 당신을 노린다,' '아는 형님', '샤펠 사단 - 어스퀘이크: 전설의 코미디',
'브레이킹 배드 무비: 엘 카미노', '사이버 지옥: n번방을 무너뜨려라', '아바타 아앙의 전설', '압꾸정,블랙핑크: 세상을 밝혀라']
# nfid_list=[81161927,81254340,80176842,81497886,81078819,81354041,70142405,81671553,81106901]

In [ ]:
설국열차_80177458,마더_81292901
jjab_seolguk_topic = ['레이턴', '봉준호', '결말', '감독', '혁명', '열차', '기대', '시즌', '꼬리', '윌포드', '멜라니',
       '시스템', '시작', '넷플릭스', '작품', '멜러니', '영화', '드라마', '민수', '설국', '기차', '이야기',
       '사건', '마지막', '반란', '아쉬움', '재밌', '공개', '한국', '주제', '괴물', '캐릭터', '모습',
       '시리즈', '양갱', '남궁', '살인', '관객', '인류', '계급', '희망', '재미', '배우', '연출',
       '메시지', '사람', '후반부', '내용', '북극곰', '송강호']
jjab_mother_topic = ['엄마', '부모', '엔딩', '마지막', '오프닝', '어머니', '자식', '마더', '완벽', '감정', '시작',
       '혜자', '잔인', '영화', '기억', '작품', '모정', '가족', '살인', '충격', '진실', '인생', '세상',
       '괴물', '김혜자', '위대', '봉준호', '강렬', '시퀀스', '모성애', '아래', '한마디', '음악', '마음',
       '관광버스', '지독', '모성', '모습', '스릴러', '테일', '흑백', '감독', '버스', '반전', '약자',
       '소름', '머더', '배우', '이해', '사이']

In [ ]:
gm = Graph.MakeGraph(mbti_data=1)
g = pickle.load(open(f'./network/final_graph/sample2/{mbti}.pickle', 'rb'))
model = Word2VecKeyedVectors.load(f'./network/model/sample2/{mbti}.model')